# Global Optimizers tests

## Scipy

#### Dual Annealing

In [1]:
from scipy.optimize import dual_annealing
import numpy as np

func = lambda x: np.sum(x*x - 10*np.cos(2*np.pi*x)) + 10*np.size(x)
lw = [-5.12] * 10
up = [5.12] * 10

In [2]:
ret = dual_annealing(func, bounds=list(zip(lw, up)), seed=1234)
print("global minimum: xmin = {0}, f(xmin) = {1:.6f}".format(ret.x, ret.fun))

global minimum: xmin = [-4.26437714e-09 -3.91699361e-09 -1.86149218e-09 -3.97165720e-09
 -6.29151648e-09 -6.53145322e-09 -3.93616815e-09 -6.55623025e-09
 -6.05775280e-09 -5.00668935e-09], f(xmin) = 0.000000


#### Simplicial Homology Global Optimization (SHGO)

In [3]:
from scipy.optimize import rosen, shgo
bounds = [(0,2), (0, 2), (0, 2), (0, 2), (0, 2)]

Bounded:

In [4]:
result = shgo(rosen, bounds)
result.x, result.fun

(array([1., 1., 1., 1., 1.]), 2.920392374190081e-18)

Empty bounds:

In [5]:
bounds = [(None, None), ]*4
result = shgo(rosen, bounds)
result.x

array([0.99999851, 0.99999704, 0.99999411, 0.9999882 ])

Egg holder function (many local minima, one global minimum):

In [6]:
def eggholder(x):
    return (-(x[1] + 47.0)
            * np.sin(np.sqrt(abs(x[0]/2.0 + (x[1] + 47.0))))
            - x[0] * np.sin(np.sqrt(abs(x[0] - (x[1] + 47.0))))
            )
bounds = [(-512, 512), (-512, 512)]

In [7]:
result = shgo(eggholder, bounds, n=30, sampling_method='sobol')
result.xl, result.funl

(array([[ 512.        ,  404.23180542],
        [ 283.07593402, -487.12566542],
        [-294.66820039, -462.01964031],
        [-105.87688985,  423.15324143],
        [-242.97923629,  274.38032063],
        [-506.25823477,    6.3131022 ],
        [-408.71980114, -156.10116115],
        [ 150.23210485,  301.31378508],
        [  91.00921872, -391.28375655],
        [ 202.8966344 , -269.38042147],
        [ 361.66625957, -106.96490692],
        [-219.40615102, -244.06022436],
        [ 151.59603137, -100.61082677]]),
 array([-959.64066272, -718.16745962, -704.80659592, -565.99778097,
        -559.78685655, -557.36868733, -507.87385942, -493.9605115 ,
        -426.48799655, -421.15571437, -419.31194957, -410.98477763,
        -202.53912972]))

In [8]:
result_2 = shgo(eggholder, bounds, n=60, iters=5, sampling_method='sobol')
len(result.xl), len(result_2.xl)

(13, 39)

#### Basin Hopping

In [9]:
from scipy.optimize import basinhopping
func = lambda x: np.cos(14.5 * x - 0.3) + (x + 0.2) * x
x0=[1.]

In [10]:
minimizer_kwargs = {"method": "BFGS"}
ret = basinhopping(func, x0, minimizer_kwargs=minimizer_kwargs, niter=200)
print("global minimum: x = %.4f, f(x0) = %.4f" % (ret.x, ret.fun))

global minimum: x = -0.1951, f(x0) = -1.0009


2D minimization problem:

In [11]:
def func2d(x):
    f = np.cos(14.5 * x[0] - 0.3) + (x[1] + 0.2) * x[1] + (x[0] + 0.2) * x[0]
    df = np.zeros(2)
    df[0] = -14.5 * np.sin(14.5 * x[0] - 0.3) + 2. * x[0] + 0.2
    df[1] = 2. * x[1] + 0.2
    return f, df

In [12]:
minimizer_kwargs = {"method":"L-BFGS-B", "jac":True}
x0 = [1.0, 1.0]
ret = basinhopping(func2d, x0, minimizer_kwargs=minimizer_kwargs, niter=200)
print("global minimum: x = [%.4f, %.4f], f(x0) = %.4f" % (ret.x[0],
                                                          ret.x[1],
                                                          ret.fun))

global minimum: x = [-0.1951, -0.1000], f(x0) = -1.0109


Prints output at every step:

In [13]:
def print_fun(x, f, accepted):
        print("at minimum %.4f accepted %d" % (f, int(accepted)))

In [14]:
np.random.seed(1)
ret = basinhopping(func2d, x0, minimizer_kwargs=minimizer_kwargs,
                   niter=10, callback=print_fun)

at minimum 0.4159 accepted 1
at minimum -0.9073 accepted 1
at minimum -0.1021 accepted 1
at minimum -0.1021 accepted 1
at minimum 0.9102 accepted 1
at minimum 0.9102 accepted 1
at minimum 2.2945 accepted 0
at minimum -0.1021 accepted 1
at minimum -1.0109 accepted 1
at minimum -1.0109 accepted 1


Bounded search:

In [15]:
class MyBounds(object):
    def __init__(self, xmax=[1.1,1.1], xmin=[-1.1,-1.1] ):
        self.xmax = np.array(xmax)
        self.xmin = np.array(xmin)
    def __call__(self, **kwargs):
        x = kwargs["x_new"]
        tmax = bool(np.all(x <= self.xmax))
        tmin = bool(np.all(x >= self.xmin))
        return tmax and tmin

In [16]:
mybounds = MyBounds()
ret = basinhopping(func2d, x0, minimizer_kwargs=minimizer_kwargs, niter=10, accept_test=mybounds)

## DEAP - Distributed Evolutionary Algorithms in Python 

In [17]:
def ackley(x):
    """
    Ackley function, 2 dimensional.
    :param x: List of parameters.
    :return: Function result, using the given x parameters.
    """
    arg1 = -0.2 * np.sqrt(0.5 * (x[0] ** 2 + x[1] ** 2))
    arg2 = 0.5 * (np.cos(2. * np.pi * x[0]) + np.cos(2. * np.pi * x[1]))
    return -20. * np.exp(arg1) - np.exp(arg2) + 20. + np.e

In [18]:
import random
from deap import algorithms, base, creator, tools
import numpy as np

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("FitnessMin", base.Fitness, weights=(1.0,))
# creator.create("Individual", list, fitness=creator.FitnessMax)
creator.create("Individual", list, fitness=creator.FitnessMin)

def evalOneMax(individual):
    x = ackley(individual)
    return_value = 1 /  np.sum(x)
    return (return_value,)

# def evalOneMax(individual):
#     return ackley(individual)

toolbox = base.Toolbox()
# toolbox.register("attr_bool", random.randint, 0, 1)
# toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=2)
toolbox.register("attr_float", random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_float, n=2)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

if __name__ == "__main__":
    pop = toolbox.population(n=10)
    
    # One liner black box:
    # algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=40, verbose=False) # OR
    
    # TODO: find all EA variations -> ~10 GA island versions
    
    # TODO: Gray/white box:    
    ngen, cxpb, mutpb = 20, 0.5, 0.2
    fitnesses = toolbox.map(toolbox.evaluate, pop)
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    for g in range(ngen):
        # TODO: island communication IO
        # TODO: individual attribs should be float in [0, 1]
        # TODO: rescale-encapsulate to NAS Ozone DNN space        
        pop = toolbox.select(pop, k=len(pop))
        
        pop = algorithms.varAnd(pop, toolbox, cxpb, mutpb)
        
        # TODO: Island swap specific individuals        
        print("len(pop)", len(pop))
        print("pop[0]", pop[0])
        if g == 3:
            print("-- Changing in generation 12 an individual")
            pop[0][0] = 0.5
            pop[0][1] = 0.5
            print("-- pop[0]", pop[0])            
            print("-- ", type(pop[0]))
            
        
        invalids = [ind for ind in pop if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalids)
        for ind, fit in zip(invalids, fitnesses):
            ind.fitness.values = fit
    
    print(tools.selBest(pop, k=1))

len(pop) 10
pop[0] [0.29539464790294234, 0.023917601833986057]
len(pop) 10
pop[0] [0.13015989009109918, 0.023917601833986057]
len(pop) 10
pop[0] [0.13015989009109918, 0.023917601833986057]
len(pop) 10
pop[0] [0.0, 0.023917601833986057]
-- Changing in generation 12 an individual
-- pop[0] [0.5, 0.5]
--  <class 'deap.creator.Individual'>
len(pop) 10
pop[0] [0.13015989009109918, 0.023917601833986057]
len(pop) 10
pop[0] [0.13015989009109918, 0.023917601833986057]
len(pop) 10
pop[0] [0.0, 0.023917601833986057]
len(pop) 10
pop[0] [0.0, 0.023917601833986057]
len(pop) 10
pop[0] [0.0, 0.0]
len(pop) 10
pop[0] [0.0, 0.023917601833986057]
len(pop) 10
pop[0] [0.0, 0.0]
len(pop) 10
pop[0] [0.0, 0.0]
len(pop) 10
pop[0] [0.0, 0.0]
len(pop) 10
pop[0] [0.0, 0.0]
len(pop) 10
pop[0] [0.0, 0.0]
len(pop) 10
pop[0] [0.0, 0.0]
len(pop) 10
pop[0] [0.0, 0.0]
len(pop) 10
pop[0] [0.0, 0.0]
len(pop) 10
pop[0] [0.0, 0.0]
len(pop) 10
pop[0] [0.0, 0.0]
[[0.0, 0.0]]


# TPE 

In [19]:
import pickle
import time
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

def objective(x):
    return {
        'loss': x ** 2,
        'status': STATUS_OK,
        # -- store other results like this
        'eval_time': time.time(),
        'other_stuff': {'type': None, 'value': [0, 1, 2]},
        # -- attachments are handled differently
        'attachments':
            {'time_module': pickle.dumps(time.time)}
        }
trials = Trials()
best = fmin(objective,
    space=hp.uniform('x', -10, 10),
    algo=tpe.suggest,
    max_evals=100,
    trials=trials)

print(best)

100%|██████████| 100/100 [00:00<00:00, 606.68it/s, best loss: 0.00023643669639453085]
{'x': 0.015376498183739068}


# Cellular automata

1d to nD index:

In [640]:
def get_1D_to_nD(index_1D, max_ranks, CA_dimensions):
    
    returning_nD_index = []   
    n = len(CA_dimensions)
    
    ca = CA_dimensions[0]
    for j in range(1, n):
        ca *= CA_dimensions[j]
        
    for i in range(1, n - 1):
        ca /= CA_dimensions[n - i]
        returning_nD_index.append(int(index_1D / ca))
        index_1D = int(index_1D % ca)

    returning_nD_index.append(int(index_1D % ca))
    returning_nD_index.append(int(index_1D / ca))
    return returning_nD_index.reverse()   

In [258]:
def get_nD_to_1D(index_nD, max_ranks, CA_dimensions):
    
    returning_1D_index = 0   
    n = len(CA_dimensions) 
    ca = CA_dimensions[0]
    for i in range(2, n):
        ca *= CA_dimensions[i - 1]
        returning_1D_index += index_nD[-(n - i)] * ca

    returning_1D_index += index_nD[-n] * CA_dimensions[1]
    returning_1D_index += index_nD[-(n - 1)]  
        
    return returning_1D_index

In [641]:
CA_dimensions = [6, 3, 4, 2, 5]
for i in range (0, 3 * 4 * 2 * 5 * 6):
    nD = get_1D_to_nD(i, max_ranks, CA_dimensions)
    print("{}: {}({})".format(nD, i,  get_nD_to_1D(nD, max_ranks, CA_dimensions)))
#     print("{}: {}({})".format(nD, i, i))

[0, 0, 0, 0, 0]: 0(0)
[0, 1, 0, 0, 0]: 1(1)
[0, 2, 0, 0, 0]: 2(2)
[0, 3, 0, 0, 0]: 3(3)
[0, 4, 0, 0, 0]: 4(4)
[0, 5, 0, 0, 0]: 5(5)
[0, 6, 0, 0, 0]: 6(6)
[0, 7, 0, 0, 0]: 7(7)
[0, 8, 0, 0, 0]: 8(8)
[0, 9, 0, 0, 0]: 9(9)
[0, 10, 0, 0, 0]: 10(10)
[0, 11, 0, 0, 0]: 11(11)
[0, 12, 0, 0, 0]: 12(12)
[0, 13, 0, 0, 0]: 13(13)
[0, 14, 0, 0, 0]: 14(14)
[0, 15, 0, 0, 0]: 15(15)
[0, 16, 0, 0, 0]: 16(16)
[0, 17, 0, 0, 0]: 17(17)
[0, 0, 1, 0, 0]: 18(18)
[0, 1, 1, 0, 0]: 19(19)
[0, 2, 1, 0, 0]: 20(20)
[0, 3, 1, 0, 0]: 21(21)
[0, 4, 1, 0, 0]: 22(22)
[0, 5, 1, 0, 0]: 23(23)
[0, 6, 1, 0, 0]: 24(24)
[0, 7, 1, 0, 0]: 25(25)
[0, 8, 1, 0, 0]: 26(26)
[0, 9, 1, 0, 0]: 27(27)
[0, 10, 1, 0, 0]: 28(28)
[0, 11, 1, 0, 0]: 29(29)
[0, 12, 1, 0, 0]: 30(30)
[0, 13, 1, 0, 0]: 31(31)
[0, 14, 1, 0, 0]: 32(32)
[0, 15, 1, 0, 0]: 33(33)
[0, 16, 1, 0, 0]: 34(34)
[0, 17, 1, 0, 0]: 35(35)
[0, 0, 2, 0, 0]: 36(36)
[0, 1, 2, 0, 0]: 37(37)
[0, 2, 2, 0, 0]: 38(38)
[0, 3, 2, 0, 0]: 39(39)
[0, 4, 2, 0, 0]: 40(40)
[0, 5, 2, 0, 0]: 41(

[0, 12, 1, 0, 2]: 318(318)
[0, 13, 1, 0, 2]: 319(319)
[0, 14, 1, 0, 2]: 320(320)
[0, 15, 1, 0, 2]: 321(321)
[0, 16, 1, 0, 2]: 322(322)
[0, 17, 1, 0, 2]: 323(323)
[0, 0, 2, 0, 2]: 324(324)
[0, 1, 2, 0, 2]: 325(325)
[0, 2, 2, 0, 2]: 326(326)
[0, 3, 2, 0, 2]: 327(327)
[0, 4, 2, 0, 2]: 328(328)
[0, 5, 2, 0, 2]: 329(329)
[0, 6, 2, 0, 2]: 330(330)
[0, 7, 2, 0, 2]: 331(331)
[0, 8, 2, 0, 2]: 332(332)
[0, 9, 2, 0, 2]: 333(333)
[0, 10, 2, 0, 2]: 334(334)
[0, 11, 2, 0, 2]: 335(335)
[0, 12, 2, 0, 2]: 336(336)
[0, 13, 2, 0, 2]: 337(337)
[0, 14, 2, 0, 2]: 338(338)
[0, 15, 2, 0, 2]: 339(339)
[0, 16, 2, 0, 2]: 340(340)
[0, 17, 2, 0, 2]: 341(341)
[0, 0, 3, 0, 2]: 342(342)
[0, 1, 3, 0, 2]: 343(343)
[0, 2, 3, 0, 2]: 344(344)
[0, 3, 3, 0, 2]: 345(345)
[0, 4, 3, 0, 2]: 346(346)
[0, 5, 3, 0, 2]: 347(347)
[0, 6, 3, 0, 2]: 348(348)
[0, 7, 3, 0, 2]: 349(349)
[0, 8, 3, 0, 2]: 350(350)
[0, 9, 3, 0, 2]: 351(351)
[0, 10, 3, 0, 2]: 352(352)
[0, 11, 3, 0, 2]: 353(353)
[0, 12, 3, 0, 2]: 354(354)
[0, 13, 3, 0, 2]: 355

[0, 15, 2, 0, 4]: 627(627)
[0, 16, 2, 0, 4]: 628(628)
[0, 17, 2, 0, 4]: 629(629)
[0, 0, 3, 0, 4]: 630(630)
[0, 1, 3, 0, 4]: 631(631)
[0, 2, 3, 0, 4]: 632(632)
[0, 3, 3, 0, 4]: 633(633)
[0, 4, 3, 0, 4]: 634(634)
[0, 5, 3, 0, 4]: 635(635)
[0, 6, 3, 0, 4]: 636(636)
[0, 7, 3, 0, 4]: 637(637)
[0, 8, 3, 0, 4]: 638(638)
[0, 9, 3, 0, 4]: 639(639)
[0, 10, 3, 0, 4]: 640(640)
[0, 11, 3, 0, 4]: 641(641)
[0, 12, 3, 0, 4]: 642(642)
[0, 13, 3, 0, 4]: 643(643)
[0, 14, 3, 0, 4]: 644(644)
[0, 15, 3, 0, 4]: 645(645)
[0, 16, 3, 0, 4]: 646(646)
[0, 17, 3, 0, 4]: 647(647)
[0, 0, 0, 1, 4]: 648(648)
[0, 1, 0, 1, 4]: 649(649)
[0, 2, 0, 1, 4]: 650(650)
[0, 3, 0, 1, 4]: 651(651)
[0, 4, 0, 1, 4]: 652(652)
[0, 5, 0, 1, 4]: 653(653)
[0, 6, 0, 1, 4]: 654(654)
[0, 7, 0, 1, 4]: 655(655)
[0, 8, 0, 1, 4]: 656(656)
[0, 9, 0, 1, 4]: 657(657)
[0, 10, 0, 1, 4]: 658(658)
[0, 11, 0, 1, 4]: 659(659)
[0, 12, 0, 1, 4]: 660(660)
[0, 13, 0, 1, 4]: 661(661)
[0, 14, 0, 1, 4]: 662(662)
[0, 15, 0, 1, 4]: 663(663)
[0, 16, 0, 1, 4]: 664

In [642]:
CA_dimensions = [3, 4, 2, 5]
for i in range (0, 3 * 4 * 2 * 5):
    nD = get_1D_to_nD(i, max_ranks, CA_dimensions)
    print("{}: {}({})".format(nD, i,  get_nD_to_1D(nD, max_ranks, CA_dimensions)))
#     print("{}: {}({})".format(nD, i, i))

[0, 0, 0, 0]: 0(0)
[0, 1, 0, 0]: 1(1)
[0, 2, 0, 0]: 2(2)
[0, 3, 0, 0]: 3(3)
[0, 4, 0, 0]: 4(4)
[0, 5, 0, 0]: 5(5)
[0, 6, 0, 0]: 6(6)
[0, 7, 0, 0]: 7(7)
[0, 8, 0, 0]: 8(8)
[0, 9, 0, 0]: 9(9)
[0, 10, 0, 0]: 10(10)
[0, 11, 0, 0]: 11(11)
[0, 0, 1, 0]: 12(12)
[0, 1, 1, 0]: 13(13)
[0, 2, 1, 0]: 14(14)
[0, 3, 1, 0]: 15(15)
[0, 4, 1, 0]: 16(16)
[0, 5, 1, 0]: 17(17)
[0, 6, 1, 0]: 18(18)
[0, 7, 1, 0]: 19(19)
[0, 8, 1, 0]: 20(20)
[0, 9, 1, 0]: 21(21)
[0, 10, 1, 0]: 22(22)
[0, 11, 1, 0]: 23(23)
[0, 0, 0, 1]: 24(24)
[0, 1, 0, 1]: 25(25)
[0, 2, 0, 1]: 26(26)
[0, 3, 0, 1]: 27(27)
[0, 4, 0, 1]: 28(28)
[0, 5, 0, 1]: 29(29)
[0, 6, 0, 1]: 30(30)
[0, 7, 0, 1]: 31(31)
[0, 8, 0, 1]: 32(32)
[0, 9, 0, 1]: 33(33)
[0, 10, 0, 1]: 34(34)
[0, 11, 0, 1]: 35(35)
[0, 0, 1, 1]: 36(36)
[0, 1, 1, 1]: 37(37)
[0, 2, 1, 1]: 38(38)
[0, 3, 1, 1]: 39(39)
[0, 4, 1, 1]: 40(40)
[0, 5, 1, 1]: 41(41)
[0, 6, 1, 1]: 42(42)
[0, 7, 1, 1]: 43(43)
[0, 8, 1, 1]: 44(44)
[0, 9, 1, 1]: 45(45)
[0, 10, 1, 1]: 46(46)
[0, 11, 1, 1]: 47(47)
[0, 

In [643]:
CA_dimensions = [3, 6, 2]
for i in range (0, 3 * 6 * 2):
    nD = get_1D_to_nD(i, max_ranks, CA_dimensions)
    print("{}: {}({})".format(nD, i,  get_nD_to_1D(nD, max_ranks, CA_dimensions)))

[0, 0, 0]: 0(0)
[0, 1, 0]: 1(1)
[0, 2, 0]: 2(2)
[0, 3, 0]: 3(3)
[0, 4, 0]: 4(4)
[0, 5, 0]: 5(5)
[0, 6, 0]: 6(6)
[0, 7, 0]: 7(7)
[0, 8, 0]: 8(8)
[0, 9, 0]: 9(9)
[0, 10, 0]: 10(10)
[0, 11, 0]: 11(11)
[0, 12, 0]: 12(12)
[0, 13, 0]: 13(13)
[0, 14, 0]: 14(14)
[0, 15, 0]: 15(15)
[0, 16, 0]: 16(16)
[0, 17, 0]: 17(17)
[0, 0, 1]: 18(18)
[0, 1, 1]: 19(19)
[0, 2, 1]: 20(20)
[0, 3, 1]: 21(21)
[0, 4, 1]: 22(22)
[0, 5, 1]: 23(23)
[0, 6, 1]: 24(24)
[0, 7, 1]: 25(25)
[0, 8, 1]: 26(26)
[0, 9, 1]: 27(27)
[0, 10, 1]: 28(28)
[0, 11, 1]: 29(29)
[0, 12, 1]: 30(30)
[0, 13, 1]: 31(31)
[0, 14, 1]: 32(32)
[0, 15, 1]: 33(33)
[0, 16, 1]: 34(34)
[0, 17, 1]: 35(35)


In [644]:
CA_dimensions = [3, 6]
for i in range (0, 3 * 6):
    nD = get_1D_to_nD(i, max_ranks, CA_dimensions)
    print("{}: {}({})".format(nD, i,  get_nD_to_1D(nD, max_ranks, CA_dimensions)))

[0, 0]: 0(0)
[0, 1]: 1(1)
[0, 2]: 2(2)
[0, 3]: 3(3)
[0, 4]: 4(4)
[0, 5]: 5(5)
[0, 6]: 6(6)
[0, 7]: 7(7)
[0, 8]: 8(8)
[0, 9]: 9(9)
[0, 10]: 10(10)
[0, 11]: 11(11)
[0, 12]: 12(12)
[0, 13]: 13(13)
[0, 14]: 14(14)
[0, 15]: 15(15)
[0, 16]: 16(16)
[0, 17]: 17(17)


In [547]:
CA_dimensions = [3, 6, 2, 3]
max_ranks = 108

for l in range (0, CA_dimensions[3]):
    for k in range (0, CA_dimensions[2]):
        for i in range(0, CA_dimensions[0]):   
            for j in range(0, CA_dimensions[1]):             
                index_nD = [i, j, k, l]
                print("{}: {}".format(index_nD, get_nD_to_1D(index_nD, max_ranks, CA_dimensions)))

[0, 0, 0, 0]: 0
[0, 1, 0, 0]: 1
[0, 2, 0, 0]: 2
[0, 3, 0, 0]: 3
[0, 4, 0, 0]: 4
[0, 5, 0, 0]: 5
[1, 0, 0, 0]: 6
[1, 1, 0, 0]: 7
[1, 2, 0, 0]: 8
[1, 3, 0, 0]: 9
[1, 4, 0, 0]: 10
[1, 5, 0, 0]: 11
[2, 0, 0, 0]: 12
[2, 1, 0, 0]: 13
[2, 2, 0, 0]: 14
[2, 3, 0, 0]: 15
[2, 4, 0, 0]: 16
[2, 5, 0, 0]: 17
[0, 0, 1, 0]: 18
[0, 1, 1, 0]: 19
[0, 2, 1, 0]: 20
[0, 3, 1, 0]: 21
[0, 4, 1, 0]: 22
[0, 5, 1, 0]: 23
[1, 0, 1, 0]: 24
[1, 1, 1, 0]: 25
[1, 2, 1, 0]: 26
[1, 3, 1, 0]: 27
[1, 4, 1, 0]: 28
[1, 5, 1, 0]: 29
[2, 0, 1, 0]: 30
[2, 1, 1, 0]: 31
[2, 2, 1, 0]: 32
[2, 3, 1, 0]: 33
[2, 4, 1, 0]: 34
[2, 5, 1, 0]: 35
[0, 0, 0, 1]: 36
[0, 1, 0, 1]: 37
[0, 2, 0, 1]: 38
[0, 3, 0, 1]: 39
[0, 4, 0, 1]: 40
[0, 5, 0, 1]: 41
[1, 0, 0, 1]: 42
[1, 1, 0, 1]: 43
[1, 2, 0, 1]: 44
[1, 3, 0, 1]: 45
[1, 4, 0, 1]: 46
[1, 5, 0, 1]: 47
[2, 0, 0, 1]: 48
[2, 1, 0, 1]: 49
[2, 2, 0, 1]: 50
[2, 3, 0, 1]: 51
[2, 4, 0, 1]: 52
[2, 5, 0, 1]: 53
[0, 0, 1, 1]: 54
[0, 1, 1, 1]: 55
[0, 2, 1, 1]: 56
[0, 3, 1, 1]: 57
[0, 4, 1, 1]: 58
[0, 5, 

In [546]:
CA_dimensions = [3, 6, 2]

for k in range (0, CA_dimensions[2]):
    for i in range(0, CA_dimensions[0]):   
        for j in range(0, CA_dimensions[1]):             
            index_nD = [i, j, k]
            print("{}: {}".format(index_nD, get_nD_to_1D(index_nD, max_ranks, CA_dimensions)))

[0, 0, 0]: 0
[0, 1, 0]: 1
[0, 2, 0]: 2
[0, 3, 0]: 3
[0, 4, 0]: 4
[0, 5, 0]: 5
[1, 0, 0]: 6
[1, 1, 0]: 7
[1, 2, 0]: 8
[1, 3, 0]: 9
[1, 4, 0]: 10
[1, 5, 0]: 11
[2, 0, 0]: 12
[2, 1, 0]: 13
[2, 2, 0]: 14
[2, 3, 0]: 15
[2, 4, 0]: 16
[2, 5, 0]: 17
[0, 0, 1]: 18
[0, 1, 1]: 19
[0, 2, 1]: 20
[0, 3, 1]: 21
[0, 4, 1]: 22
[0, 5, 1]: 23
[1, 0, 1]: 24
[1, 1, 1]: 25
[1, 2, 1]: 26
[1, 3, 1]: 27
[1, 4, 1]: 28
[1, 5, 1]: 29
[2, 0, 1]: 30
[2, 1, 1]: 31
[2, 2, 1]: 32
[2, 3, 1]: 33
[2, 4, 1]: 34
[2, 5, 1]: 35


In [545]:
CA_dimensions = [3, 6]
for i in range(0, CA_dimensions[0]):
    for j in range(0, CA_dimensions[1]):
        index_nD = [i, j]
        print("{}: {}".format(index_nD, get_nD_to_1D(index_nD, max_ranks, CA_dimensions)))

[0, 0]: 0
[0, 1]: 1
[0, 2]: 2
[0, 3]: 3
[0, 4]: 4
[0, 5]: 5
[1, 0]: 6
[1, 1]: 7
[1, 2]: 8
[1, 3]: 9
[1, 4]: 10
[1, 5]: 11
[2, 0]: 12
[2, 1]: 13
[2, 2]: 14
[2, 3]: 15
[2, 4]: 16
[2, 5]: 17


In [255]:
CA_dimensions = [3, 2, 2, 3, 2, 3]
max_ranks = 108 * 2

for y in range (0, CA_dimensions[5]):
    for x in range (0, CA_dimensions[4]):
        for l in range (0, CA_dimensions[3]):
            for k in range (0, CA_dimensions[2]):
                for i in range(0, CA_dimensions[0]):   
                    for j in range(0, CA_dimensions[1]):             
                        index_nD = [i, j, k, l, x, y]
                        print("{}: {}".format(index_nD, get_nD_to_1D(index_nD, max_ranks, CA_dimensions)))

[0, 0, 0, 0, 0, 0]: 0
[0, 1, 0, 0, 0, 0]: 1
[1, 0, 0, 0, 0, 0]: 2
[1, 1, 0, 0, 0, 0]: 3
[2, 0, 0, 0, 0, 0]: 4
[2, 1, 0, 0, 0, 0]: 5
[0, 0, 1, 0, 0, 0]: 6
[0, 1, 1, 0, 0, 0]: 7
[1, 0, 1, 0, 0, 0]: 8
[1, 1, 1, 0, 0, 0]: 9
[2, 0, 1, 0, 0, 0]: 10
[2, 1, 1, 0, 0, 0]: 11
[0, 0, 0, 1, 0, 0]: 12
[0, 1, 0, 1, 0, 0]: 13
[1, 0, 0, 1, 0, 0]: 14
[1, 1, 0, 1, 0, 0]: 15
[2, 0, 0, 1, 0, 0]: 16
[2, 1, 0, 1, 0, 0]: 17
[0, 0, 1, 1, 0, 0]: 18
[0, 1, 1, 1, 0, 0]: 19
[1, 0, 1, 1, 0, 0]: 20
[1, 1, 1, 1, 0, 0]: 21
[2, 0, 1, 1, 0, 0]: 22
[2, 1, 1, 1, 0, 0]: 23
[0, 0, 0, 2, 0, 0]: 24
[0, 1, 0, 2, 0, 0]: 25
[1, 0, 0, 2, 0, 0]: 26
[1, 1, 0, 2, 0, 0]: 27
[2, 0, 0, 2, 0, 0]: 28
[2, 1, 0, 2, 0, 0]: 29
[0, 0, 1, 2, 0, 0]: 30
[0, 1, 1, 2, 0, 0]: 31
[1, 0, 1, 2, 0, 0]: 32
[1, 1, 1, 2, 0, 0]: 33
[2, 0, 1, 2, 0, 0]: 34
[2, 1, 1, 2, 0, 0]: 35
[0, 0, 0, 0, 1, 0]: 36
[0, 1, 0, 0, 1, 0]: 37
[1, 0, 0, 0, 1, 0]: 38
[1, 1, 0, 0, 1, 0]: 39
[2, 0, 0, 0, 1, 0]: 40
[2, 1, 0, 0, 1, 0]: 41
[0, 0, 1, 0, 1, 0]: 42
[0, 1, 1, 0, 1, 0]: 4

In [256]:
CA_dimensions = [3, 6, 2, 3, 2]
max_ranks = 108 * 2

for x in range (0, CA_dimensions[4]):
    for l in range (0, CA_dimensions[3]):
        for k in range (0, CA_dimensions[2]):
            for i in range(0, CA_dimensions[0]):   
                for j in range(0, CA_dimensions[1]):             
                    index_nD = [i, j, k, l, x]
                    print("{}: {}".format(index_nD, get_nD_to_1D(index_nD, max_ranks, CA_dimensions)))

[0, 0, 0, 0, 0]: 0
[0, 1, 0, 0, 0]: 1
[0, 2, 0, 0, 0]: 2
[0, 3, 0, 0, 0]: 3
[0, 4, 0, 0, 0]: 4
[0, 5, 0, 0, 0]: 5
[1, 0, 0, 0, 0]: 6
[1, 1, 0, 0, 0]: 7
[1, 2, 0, 0, 0]: 8
[1, 3, 0, 0, 0]: 9
[1, 4, 0, 0, 0]: 10
[1, 5, 0, 0, 0]: 11
[2, 0, 0, 0, 0]: 12
[2, 1, 0, 0, 0]: 13
[2, 2, 0, 0, 0]: 14
[2, 3, 0, 0, 0]: 15
[2, 4, 0, 0, 0]: 16
[2, 5, 0, 0, 0]: 17
[0, 0, 1, 0, 0]: 18
[0, 1, 1, 0, 0]: 19
[0, 2, 1, 0, 0]: 20
[0, 3, 1, 0, 0]: 21
[0, 4, 1, 0, 0]: 22
[0, 5, 1, 0, 0]: 23
[1, 0, 1, 0, 0]: 24
[1, 1, 1, 0, 0]: 25
[1, 2, 1, 0, 0]: 26
[1, 3, 1, 0, 0]: 27
[1, 4, 1, 0, 0]: 28
[1, 5, 1, 0, 0]: 29
[2, 0, 1, 0, 0]: 30
[2, 1, 1, 0, 0]: 31
[2, 2, 1, 0, 0]: 32
[2, 3, 1, 0, 0]: 33
[2, 4, 1, 0, 0]: 34
[2, 5, 1, 0, 0]: 35
[0, 0, 0, 1, 0]: 36
[0, 1, 0, 1, 0]: 37
[0, 2, 0, 1, 0]: 38
[0, 3, 0, 1, 0]: 39
[0, 4, 0, 1, 0]: 40
[0, 5, 0, 1, 0]: 41
[1, 0, 0, 1, 0]: 42
[1, 1, 0, 1, 0]: 43
[1, 2, 0, 1, 0]: 44
[1, 3, 0, 1, 0]: 45
[1, 4, 0, 1, 0]: 46
[1, 5, 0, 1, 0]: 47
[2, 0, 0, 1, 0]: 48
[2, 1, 0, 1, 0]: 49
[2, 2, 0, 

In [114]:
CA_dimensions = [3, 6, 2]
max_ranks = 27
index_nD = [1, 3, 1]
returned_1D = get_nD_to_1D(index_nD, max_ranks, CA_dimensions)
print("index_nD: {}, returned_1D: {}".format(index_nD, returned_1D))

CA_dimensions = [3, 6]
max_ranks = 18
index_nD = [2, 2]
returned_1D = get_nD_to_1D(index_nD, max_ranks, CA_dimensions)
print("index_nD: {}, returned_1D: {}".format(index_nD, returned_1D))

CA_dimensions = [3, 6]
max_ranks = 18
index_nD = [1, 2]
returned_1D = get_nD_to_1D(index_nD, max_ranks, CA_dimensions)
print("index_nD: {}, returned_1D: {}".format(index_nD, returned_1D))


index_nD: [1, 3, 1], returned_1D: 27
index_nD: [2, 2], returned_1D: 14
index_nD: [1, 2], returned_1D: 8


In [68]:
def get_1D_to_nD(index_1D, max_ranks, CA_dimensions):
    
    returning_nD_index = []
    CA_multiplications = []
    
    multiplied = CA_dimensions[0]
    for i in range(1, len(CA_dimensions) - 1):
        dimensions_size = CA_dimensions[i]
        multiplied *=  dimensions_size
        CA_multiplications.append(multiplied)      
    CA_multiplications.reverse()
    CA_multiplications.append(CA_dimensions[1]) 
    print(CA_multiplications) 
    return returning_nD_index

In [70]:
CA_dimensions = [3, 6, 2]
max_ranks = 27
index_1D = 27
index_nD = get_1D_to_nD(index_1D, max_ranks, CA_dimensions)

[18, 6]


In [51]:
CA_dimensions = [3, 6, 2, 3]
max_ranks = 108

index_1D = 107
index_nD = get_1D_to_nD(index_1D, max_ranks, CA_dimensions)

[36, 18, 6]
